<a href="https://colab.research.google.com/github/hongwon2/BigQuery-Practice/blob/main/BigQuery%EB%A1%9C_%EB%B6%84%EC%84%9D%ED%95%9C_Hongwon's_Data_%EB%B8%94%EB%A1%9C%EA%B7%B8_%5B2%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'liquid-terra-362502' # Project ID inserted based on the query results selected to explore
location = 'asia-northeast3' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## 접속 요일과 시간대 알아보기

In [ ]:
job = client.get_job('bquxjob_3e5f081c_18471802eaa')
print(job.query)

with time_stamp as (
select 
  timestamp_seconds(cast(cast(event_timestamp as INT64)/1000000 as INT64)) + interval '9' hour as event_time_stamp
from `analytics_310464334.events_*`
where _TABLE_SUFFIX between '20221009'and'20221112' and event_name = 'session_start'
order by event_time_stamp
),
access_log_with_dow as (
select 
  event_time_stamp,
  --일요일(0) ~ 토요일(6)
  extract(dayofweek from timestamp(event_time_stamp)) - 1 as dow, 
  --시간을 초로 변환
  cast(substring(cast(event_time_stamp as STRING),12,2) as INT64) * 60 * 60
  + cast(substring(cast(event_time_stamp as STRING),15,2) as INT64) * 60
  + cast(substring(cast(event_time_stamp as STRING),18,2) as INT64)
  as whole_seconds,
  --시간 간격 (1시간(3600초)) 지정
  60 * 60 as interval_seconds
from time_stamp
),
access_log_with_floor_seconds as (
select 
  event_time_stamp,
  dow,
  cast((floor(whole_seconds/interval_seconds) * interval_seconds) as INT64) as floor_seconds
from access_log_with_dow
),
access_log_with_index as (
select
  event_time_st

In [ ]:
job = client.get_job('bquxjob_3e5f081c_18471802eaa')
results = job.to_dataframe()
results

,index_time,sun,mon,tue,wed,thu,fri,sat
0,00:00:00,5,6,8,8,14,6,10
1,01:00:00,2,1,3,7,10,5,4
2,02:00:00,0,0,1,8,5,1,9
3,03:00:00,1,2,2,2,2,0,6
4,04:00:00,0,3,0,1,6,1,1
5,05:00:00,3,1,0,0,0,3,2
6,06:00:00,1,4,1,0,0,1,5
7,07:00:00,0,2,3,2,2,3,3
8,08:00:00,0,3,1,4,8,3,5
9,09:00:00,1,10,7,13,10,13,3
